# Prompt Engineering

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

In this demo notebook, we are going to explore some of the PROMPT Engineering Techniques for better model output generation

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/d4/e8/3ccebf1cb78702beb8bb2e535525ff350d0f597ff794baeeab5206800a2c/boto3-1.34.1-py3-none-any.whl.metadata
  Using cached boto3-1.34.1-py3-none-any.whl.metadata (6.6 kB)
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/4c/89/225b9353313e73705738740a60449787f0431b1c68c4c68ccff0c2056a2b/awscli-1.32.1-py3-none-any.whl.metadata
  Using cached awscli-1.32.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/74/ac/1ebddbe1f9f23a36e4b0ff2b0f2c78284ccfd4be6a7148fdb1605c7e5913/botocore-1.34.1-py3-none-any.whl.metadata
  Using cached botocore-1.34.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.10.0,>=0.9.0 from https://files.pythonhosted.org/packages/fd/fb/46eda754e80fa2

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install other packages.

Here are links for some packages you might not be familiar with:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [sqlalchemy](https://www.sqlalchemy.org/), SQLAlchemy is the Python SQL toolkit 
- [Pinecone](http://pinecone.io), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files
- [IPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), for interactive UI widgets in the notebook
- [NeMo-Guardrails](https://github.com/NVIDIA/NeMo-Guardrails) an open-source toolkit for easily adding programmable guardrails to LLM-based conversational systems


### text

In [3]:
%pip install --quiet \
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client \
    apache-beam \
    datasets \
    tiktoken \
    "ipywidgets>=7,<8" \
    matplotlib



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### agents

In [4]:
%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    pysqlite3 \
    google-search-results


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nemoguardrails 0.5.0 requires langchain==0.0.251, but you have langchain 0.0.350 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### entity extraction

In [5]:
%pip install --quiet beautifulsoup4



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### image

In [6]:
%pip install --quiet "pillow>=9.5,<10"



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### guardrails

In [7]:
%%bash
apt-get update && apt-get install g++ -y


Get:1 http://security.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Hit:2 http://deb.debian.org/debian bullseye InRelease
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Fetched 92.4 kB in 0s (264 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:10.2.1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:

# %pip install -qU --no-cache-dir nemoguardrails==0.5.0

# %pip install -qU "faiss-cpu>=1.7,<2" \
#                       "langchain==0.0.309" \
#                       "pypdf>=3.8,<4" \
#                       "ipywidgets>=7,<8"


### Restart Kernel 

In [9]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

Depending on your environment, you might need to customize the setup when creating your Bedrock service client. To help with this, we've provided a `get_bedrock_client()` utility method that supports passing in different options. You can find the implementation in [../utils/bedrock.py](../utils/bedrock.py)

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).

The `get_bedrock_client()` method accepts `runtime` (default=True) parameter to return either `bedrock` or `bedrock-runtime` client.

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### A note about `langchain`

The Bedrock classes provided by `langchain` create a Bedrock boto3 client by default. To customize your Bedrock configuration, we recommend to explicitly create the Bedrock client using the method below, and pass it to the [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) class instantiation method using `client=boto3_bedrock`

In [10]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::067564772063:role/Crossaccountbedrock"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)


Create new client
  Using region: us-east-1
  Using role: arn:aws:iam::067564772063:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


#### Validate the connection

We can check the client works by trying out the `list_foundation_models()` method, which will tell us all the models available for us to use 

In [11]:
boto3_bedrock.list_foundation_models()


{'ResponseMetadata': {'RequestId': 'e6cfd3ad-5129-495b-9078-ef3b275314a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Dec 2023 15:09:52 GMT',
   'content-type': 'application/json',
   'content-length': '17836',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e6cfd3ad-5129-495b-9078-ef3b275314a0'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

---

## `InvokeModel` body and output

The `invoke_model()` method of the Amazon Bedrock runtime client (`InvokeModel` API) will be the primary method we use for most of our Text Generation and Processing tasks - whichever model we're using.

Although the method is shared, the format of input and output varies depending on the foundation model used - as described below:

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

---

## Try out the Claude Model

With some theory out of the way, let's see the models in action! Run the cells below to see basic, synchronous example invocations for each model:

In [12]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


Create new client
  Using region: us-east-1
  Using role: arn:aws:iam::067564772063:role/Crossaccountbedrock ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [13]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""


In [14]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 Here is a suggested blog post on making strong business decisions as a leader:

Making Strong Business Decisions as a Leader  

As a leader in your organization, some of your most important responsibilities center around making key business decisions. These decisions can have a significant impact on the direction and success of your company. So how can you make strong, well-thought out decisions? Here are some tips:

•Gather input from your team. Get multiple perspectives on the issues and options. Your team members likely have insights and experience that can inform your thinking. Listen with an open mind to their feedback. 

•Do your research. Make sure you understand the full context and have all the relevant data and facts. Look at trends in the industry and marketplace. The more informed you are, the better your decision making will be.

•Consider all options carefully. Don't narrow in on one choice too quickly. Explore multiple alternatives to find the solution that best solves 

## Prompt Engineering - Basic Techiques

In this section, we are going to explore some basic prompting engineering techniques.

---

### Zero-shot prompting
Zero Shot prompting describes the technique where we present a task to an LLM without giving it further examples. We therefore, expect it to perform the task without getting a prior look at a “shot” at the task. Hence, “zero-shot” prompting. Modern LLMs demonstrate remarkable zero-shot performance and a positive correlation can be drawn between model size and zero-shot performance.

In [15]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: You are a customer service agent tasked with classifying emails by type. Please output your answer and then justify your classification. How would you categorize this email? 
<email>
I would like to know how I can contribute to my retirement account? Any additional resources to read about IRA would be helpful
</email> 
Please explanation for your choice of answer

The categories are: 
(A) IRA 
(B) 529 Plan
(C) Cash Management
(D) Youth Account

Assistant:
"""


In [16]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 A. IRA

Justification: The email specifically asks about IRA (Individual Retirement Account) contributions and mentions IRA by name. The customer is looking for information on how to contribute to their IRA and for additional resources to learn more about IRAs. The other options do not match as closely based on the content and specifics of the email.


---

### Few-shot prompting
Giving the model more information about the tasks at hand via examples is called Few-Shot Prompting. It can be used for in-context learning by providing examples of the task and the desired output. We can therefore condition the model on the examples to follow the task guidance more closely.

In [17]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Provide a brief financial summary for the following companies: 

Microsoft: Microsoft reported record revenue of $51.9 billion in its latest quarterly earnings report, a 20% increase year-over-year. Its net income rose to $16.7 billion, up 21% compared to the previous year. Microsoft's revenue growth was driven by its cloud computing business Azure and Office 365 productivity tools.

Amazon: In its most recent quarterly results, Amazon reported net sales of $96.1 billion, an increase of 15% compared to the same period last year. Amazon's net income decreased to $2.9 billion, down 58% year-over-year, as the company faced rising costs due to inflation and supply chain issues. Amazon Web Services, its cloud computing segment, continued to see strong growth with sales up 33%.

Now provide a brief financial summary for Apple Inc. based on the following key facts:
- Revenue for the last 12 months: $260 billion 
- Net income: $55 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

Assistant:
"""


In [18]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 Here is a brief financial summary for Apple Inc.:

Apple reported revenue of $260 billion over the last 12 months. Net income totaled $55 billion, reflecting a healthy profit margin. Apple has a strong balance sheet with $321 billion in total assets. As of February 2022, Apple's market capitalization stands at $2.2 trillion, making it the largest company in the world by market cap. Revenue and profits continue to be driven by strong sales of the iPhone, as well as services like Apple Music and Apple Pay.


---

### Chain of Thought prompting
Chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding. The main idea of CoT is that by showing the LLM some few shot exemplars where the reasoning process is explained in the exemplars, the LLM will also show the reasoning process when answering the prompt. This explanation of reasoning often leads to more accurate results.

In [19]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Context: Apple Inc. key facts:
- Revenue for the last 12 months: $260 billion 
- Net income: $55 billion
- Total assets: $321 billion
- Market capitalization: $2.2 trillion

Human: Think step-by-step for providing a financial summary for Apple Inc. based on the information shared in the Context before.

Include your thoughts in the <thinking></thinking> XML tags and the actual summary as an Answer.

Assistant:
"""


In [20]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 <thinking> 
To provide a financial summary, I first need to understand the key metrics and compare them to Apple's financial performance. The key metrics are:
- Revenue: Shows the total amount of money Apple made from its business operations. $260 billion is very high, indicating strong growth.
- Net income: Measures the profit Apple earned. $55 billion is a high net income, showing Apple is very profitable. 
- Total assets: Represents everything Apple owns and is used to generate its revenue. $321 billion in assets shows Apple has a strong financial position.  
- Market capitalization: Calculates the total value of Apple's shares in the stock market. $2.2 trillion market cap means investors see huge potential for growth and the company is very valuable.
</thinking>

Answer: 

Financial Summary for Apple Inc.

- Revenue: $260 billion. Apple reported strong revenue growth over the last year from its products and services. 
- Net income: $55 billion. Apple is a highly profitable company

## Prompt Engineering - Advanced Techiques

In this section, we are going to explore some basic prompting engineering Advanced techniques.

---

### Role Prompting
You can use role prompting as a way to get Claude to emulate certain styles in writing . Speak in a certain voice or guide the complexity of its answers. Role prompting also make Claude better at prompting math or logic tasks.

In [21]:
# PROMPT Without ROLE
prompt_data = """Human: Please explain the impact of greenhouse gases.

Assistant:
"""

In [22]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

 Greenhouse gases like carbon dioxide and methane trap heat in the atmosphere, causing it to warm. This is known as the greenhouse effect. The major impacts of increased greenhouse gases include:

• Rising global temperatures: As greenhouse gases trap more heat, the temperature of the planet increases over time. Average global temperatures have increased about 1 degree Celsius over the past 140 years, according to the latest science.

• Changes in weather patterns: Increased temperatures are altering weather patterns and climate systems. Some areas are seeing changes in rainfall amounts and distribution, worsening droughts, more frequent heat waves, changes in the timing of seasons, etc. These changes can lead to reduced agricultural production, water shortages, wildfires, etc.

• Rising sea levels: As the planet warms, sea levels are rising for two main reasons:

1. Ice sheets and glaciers are melting: Large ice sheets and glaciers around the world are melting due to rising temperatur

In [23]:
# PROMPT WITH ROLE
prompt_data = """Human: You are a highly experienced climate scientist. Please explain the impact of greenhouse gases.

Assistant:
"""

In [24]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

 Greenhouse gases, such as carbon dioxide and methane, trap heat in the atmosphere, causing it to warm. This is known as the greenhouse effect. The more greenhouse gases are emitted into the atmosphere, the more they accumulate and the more heat they trap. This leads to an increase in average surface temperatures on Earth and changes in weather patterns and sea levels.

The main impacts of increased greenhouse gases include:

•Rising global temperatures - Average surface temperatures have increased about 1 degree Celsius over the past 140 years according to the latest IPCC report. The past few decades have likely been the warmest in the last 400 years according to climate reconstructions.

•Rising sea levels - As the climate continues to warm, glaciers and ice sheets melt and oceans expand. This causes sea levels to rise. Sea level has risen 8 to 9 inches since 1900 and the rate of rise is accelerating according to recent studies. Higher sea levels lead to coastal flooding and erosion.